In [31]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
import os.path
import time, itertools, re
import importlib
from collections import Counter
import scipy as sc
import miceforest as mf
from sklearn.metrics import make_scorer, balanced_accuracy_score, confusion_matrix, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.inspection import permutation_importance
from HypOpt import lowest_cwacc
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from EDdataset_GGZ import colsTypeCast    
#df_all_combo=pd.read_csv(fileloc_data+'maskedDAIsy7_MainDec_EDEQ_SQ48_MHCSF_Honos_Lav_Visit1.tsv', sep='\t', decimal=',')
#df_adapted_combo, colsExtracted, subscales=colsTypeCast(df_all_combo)
#adapted_combo_cols=np.setdiff1d(colsExtracted,
#        ['ED_Codes','EDtype', 'SQ48-Score', 'MHCSF-Score', 'Lav-Score']+list(subscales['Honos']))
#print(adapted_combo_cols)

/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/


In [32]:
%load_ext autoreload
from GetDataReady import getDataNormalized_Interact, getDataNormalized

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Classification

In [33]:
choice_dict={1.0: 'Core-DT, 5Cls', 1.1: 'Core-DT, only ED', 1.2: 'Core-DT, 3 ED and Others',
    2.0: 'Core-DT-EDEQ subscale, 5Cls', 2.1: 'Core-DT-EDEQ subscale, only ED', 2.2: 'Core-DT-EDEQ subscale, 3 ED and Others',
    3.0: 'Core, 5Cls', 3.1: 'Core, only ED', 3.2: 'Core, 3 ED and Others'}
savepicklpath='%s/pickles/'%(os.getcwd())
dataset_type='allCombiAllExps'#'Core_DT'#save_dict[choice]
model_fname='cls-%s.pkl'%dataset_type
modelsClassify1='%s%s'%(savepicklpath, model_fname)
save_dict={1.0: 'Core-DT-5Cls', 1.1: 'Core-DT-onlyED', 1.2: 'Core-DT-3ED-Others',
    2.0: 'Core-DT-EDEQ-5Cls', 2.1: 'Core-DT-EDEQ-only-ED', 2.2: 'Core-DT-EDEQ-3ED-Others',
    3.0: 'Core-5Cls', 3.1: 'Core-onlyED', 3.2: 'Core-3ED-Others'}
dataset_types={}
%autoreload 2
from HypOpt import GridSearchClassifiers, GridSearch_LVQ
sampling_strategy='not majority'
param_grid={'RF_Max_Features': [5,7,10, 20,25], 'RF_n_Trees':[100, 300, 500], 'RF_min_leaf':5,
           'scorer_name':'Lowest_CWA', 'LRL1_C': np.array([0.001, 0.01, 0.1, 1])}
for key, val in choice_dict.items():
    dataset_types[key]=getDataNormalized(key)
    X,Y=dataset_types[key]['zXtrain'], dataset_types[key]['Ytrain']  
    if key>=3:        
        if X.isnull().sum().sum()>0:
            X,Y=dataset_types[key]['mice_zXtrain'], dataset_types[key]['Ytrain'] 
    if key in [1.0,2.0,3.0]:
        print(X.columns)

1.0: For dataset with Core, DT, and 5 classes (Ndims=24):

Index(['DT-BMI', 'DT-Disorder_Duration_Yrs', 'DT-IND_BDL_CMD',
       'DT-IND_OCD_CMD', 'DT-IND_depression_CMD', 'DT-IND_others',
       'DT-IND_prev_spec_int_wo_eff', 'DT-num_prev_routes',
       'Lav-Dissatisfaction_body', 'Lav-Negative_appraisal_body',
       'Lav-Unfamiliarity_with_body', 'MHCSF-Emotional_Well-being',
       'MHCSF-Psychological_Well-being', 'MHCSF-Social_Well-being', 'Main-Age',
       'SQ48-Agoraphobia', 'SQ48-Anxiety', 'SQ48-Cognitive_Complaints',
       'SQ48-Depression', 'SQ48-Hostility', 'SQ48-Social_phobia',
       'SQ48-Somatic_Complaints', 'SQ48-Vitality',
       'SQ48-Work_related_complaints'],
      dtype='object')
1.1: For dataset with Core, DT and only ED classes:

1.2: For dataset with Core, DT and 3 ED classes and Others:

2.0: For dataset with Core, DT, EDEQ subscales and all 5 classes (Ndim=27):

Index(['DT-BMI', 'DT-Disorder_Duration_Yrs', 'DT-IND_BDL_CMD',
       'DT-IND_OCD_CMD', 'DT-IND

In [4]:
%autoreload 2
from HypOpt import GridSearchClassifiers, GridSearch_LVQ
sampling_strategy='not majority'
param_grid={'RF_Max_Features': [5,7,10, 20,25], 'RF_n_Trees':[100, 300, 500], 'RF_min_leaf':5,
           'scorer_name':'Lowest_CWA', 'LRL1_C': np.array([0.001, 0.01, 0.1, 1])}
dataset_types={}
for key, val in choice_dict.items():
    dataset_types[key]=getDataNormalized(key)
    X,Y=dataset_types[key]['zXtrain'], dataset_types[key]['Ytrain']  
    if key<3:        
        if X.isnull().sum().sum()>0:
            X,Y=dataset_types[key]['mice_zXtrain'], dataset_types[key]['Ytrain']  
    GridSearchClassifiers(X, Y,sampling_strategy, param_grid)

1.0: For dataset with Core, DT, and 5 classes (Ndims=24):

RF: 
    n_Trees  Max_Features  Lowest_CWA
2      500             5    0.515159
4      300             7    0.512460
LogLASSO: 
      C  Lowest_CWA
3  1.0    0.376111
2  0.1    0.253254
KNN: 
    K Dist Metric  Lowest_CWA
0  3   minkowski    0.452381
3  3      cosine    0.440317
LDA: 
   Solver  Lowest_CWA
0    svd    0.389444
1   lsqr    0.389444
Linear SVM: 
        C  Lowest_CWA
5  100.0    0.504365
4   10.0    0.493730
RBF SVM: 
         C  Gamma  Lowest_CWA
32  100.0   0.01    0.624048
26   10.0   0.01    0.532143
1.1: For dataset with Core, DT and only ED classes:

RF: 
    n_Trees  Max_Features  Lowest_CWA
2      500             5    0.553968
0      100             5    0.543095
LogLASSO: 
      C  Lowest_CWA
3  1.0    0.499524
2  0.1    0.316270
KNN: 
    K Dist Metric  Lowest_CWA
3  3      cosine    0.517302
0  3   minkowski    0.476032
LDA: 
   Solver  Lowest_CWA
2  eigen    0.525238
0    svd    0.507063
Linear SVM: 


In [97]:
import pickle
fname_pkl1, fname_pkl2='Hyp_LVQ.pkl', 'Hyp_LVQ2.pkl'
savepicklpath='%s/pickles/'%(os.getcwd())
LVQModels1, LVQModels2='%s%s'%(savepicklpath, fname_pkl1), '%s%s'%(savepicklpath, fname_pkl2)
choice, rows=1.2,[]
if os.path.exists(LVQModels1):
        LVQClassifiers1 = pickle.load(open(LVQModels1, "rb"))#LVQClassifiers2 = pickle.load(open(LVQModels2, "rb"))print('LVQ1: ',LVQClassifiers1.keys())#print('LVQ2: ',LVQClassifiers2.keys())
print('Choice is ',choice)
print(LVQClassifiers1[choice]['GMLVQ'].head(1),'\n', LVQClassifiers1[choice]['LGMLVQ'][cols_disp].head(1))
g_prot_per_cls=LVQClassifiers1[choice]['GMLVQ']['Prot per cls'].to_numpy()[0]
l_prot_per_cls=LVQClassifiers1[choice]['LGMLVQ']['Prot per cls'].to_numpy()
print('Matching LGMLVQ is:')
for idx in range(0,10):
    if np.sum(l_prot_per_cls[idx]==g_prot_per_cls)==len(g_prot_per_cls):
        rows.append(idx)
cols_disp=['Prot per cls','Solver', 'Num Component','Activation type', 'Lowest_CWA']
print(LVQClassifiers1[choice]['LGMLVQ'][cols_disp].iloc[rows])

Choice is  1.2
     Prot per cls Solver  Num Component Activation type  Reg term  Lowest_CWA
307  [3, 2, 1, 1]   adam             15           swish     0.001    0.699242 
     Prot per cls Solver  Num Component Activation type  Lowest_CWA
34  [3, 3, 2, 1]  lbfgs              7        identity    0.729293
Matching LGMLVQ is:
     Prot per cls Solver  Num Component Activation type  Lowest_CWA
200  [3, 2, 1, 1]    sgd             20           swish    0.721717
198  [3, 2, 1, 1]  lbfgs             15           swish    0.719949
202  [3, 2, 1, 1]  lbfgs             20           swish    0.718939


In [15]:
iters, nreps=5,10#pipeClassifiers=[]
use_permutation_imp=['KNN','LDA','QDA','LSVC','RSVC']
nTrees, max_ftrs=np.array([500,500,100, 300,500,500, 500,300,300]), np.array([5,5,7, 10,7,5, 7,5,7])
C_LR,knn=np.array([1,1,1, 1,1,1, 1,1,1]), np.array([3,3,3, 3,3,3, 3,3,3])
knn_metric=['minkowski','cosine','cosine', 'cosine','cosine','cosine', 'cosine','cosine','cosine']
lda_solver=['svd','eigen','svd', 'svd','svd','eigen',  'svd', 'svd','svd']
C_Lin, C_RBF=np.array([100,100,1, 10,100,1, 100,10,100]),np.array([100,100,100, 100,100,100, 10, 100, 10])
svd_gamma=np.array([0.01,0.01, 0.01, 0.01, 0.01, 0.01, 1, 0.01, 1])
s_glob={1.0:'adam',1.1:'lbfgs',1.2:'adam', 2.0:'lbfgs',2.1:'lbfgs',2.2:'lbfgs',3.0:'lbfgs',3.1:'lbfgs',3.2:'lbfgs'}
s_loc1={1.0:'lbfgs',1.1:'lbfgs',1.2:'lbfgs',2.0:'lbfgs',2.1:'lbfgs',2.2: 'lbfgs', 3.0:'lbfgs',3.1:'adam',3.2:'lbfgs'}
s_loc2={1.0:'lbfgs',1.1:'lbfgs',1.2:'sgd',2.0:'lbfgs',2.1:'lbfgs',2.2: 'lbfgs', 3.0:'lbfgs',3.1:'adam',3.2:'lbfgs'}
act_g={1.0:'swish', 1.1:'swish',1.2:'swish',2.0:'swish',2.1:'identity', 2.2:'swish',
       3.0:'swish',3.1:'swish',3.2:'swish'}
act_l1={1.0:'swish',1.1:'swish', 1.2:'identity', 2.0:'identity',2.1:'swish',2.2:'identity',
        3.0:'identity',3.1:'swish',3.2: 'identity' }
act_l2={1.0:'swish',1.1:'identity', 1.2:'swish', 2.0:'identity',2.1:'swish',2.2:'swish',
        3.0:'swish',3.1:'swish',3.2: 'identity' }
rel_comp_gmlvq={1.0:'20', 1.1:'7', 1.2:'15', 2.0:'5', 2.1:'20', 2.2:'15', 3.0:'10', 3.1:'7', 3.2:'15'}
rel_comp_lgmlvq1={1.0:'10', 1.1:'15', 1.2:'7', 2.0:'20', 2.1:'7', 2.2:'20', 3.0:'5', 3.1:'15', 3.2:'7'}
rel_comp_lgmlvq2={1.0:'10', 1.1:'20', 1.2:'20', 2.0:'10', 2.1:'7', 2.2:'20', 3.0:'5', 3.1:'15', 3.2:'7'}
g_num_prots_all={1.0:np.array([2, 2, 2, 1, 1]),1.1:np.array([3, 2, 1, 1]),1.2:np.array([3, 2, 1, 1]),
        2.0:np.array([3, 2, 2, 1, 1]), 2.1:np.array([3,2, 2, 1]), 2.2:np.array([3, 2, 1, 1]),
        3.0:np.array([3, 2, 2, 1, 1]), 3.1:np.array([3, 2, 1, 1]),3.2:np.array([3,3,2,1])}
l_num_prots_all1={1.0:np.array([2, 2, 2, 1, 1]),1.1:np.array([3, 3, 2, 1]),1.2:np.array([3, 3, 2, 1]),
        2.0:np.array([3, 3, 3, 1, 1]),2.1:np.array([2, 1, 2, 1]), 2.2:np.array([3, 2, 2, 1]),
        3.0:np.array([3,3,3, 1, 1]), 3.1:np.array([3,3, 2, 1]), 3.2:np.array([3,3, 2, 1])}
l_num_prots_all2=g_num_prots_all.copy()
reg_comp, rel_loc=np.array([0.001,0, 0.001, 0.001,0,0.01, 0,0.01, 0.0]),'class'
pipeClassifiers_all, c={},0

In [16]:
for keyD, val in choice_dict.items():
    dataset_type=save_dict[keyD]#'Core_DT'#save_dict[choice]
    model_fname='%s_clf2.pkl'%dataset_type
    modelsClassify='%s%s'%(savepicklpath, model_fname)
    modelsClassify_permut='%s%s_%s.pkl'%(savepicklpath,dataset_type, 'FtrImp')    
    Y=dataset_types[keyD]['Ytrain']  
    if keyD>=3:        
        X=dataset_types[keyD]['zXtrain']  
    else:
        X=dataset_types[keyD]['mice_zXtrain']
    if ~(os.path.exists(modelsClassify)) | ~(os.path.exists(modelsClassify_permut)):
        clf_all, temp, permut_model, all_prots={},{}, {},{}
        for iter in range(iters):            
            pipeRF=Pipeline(steps=[('RF', RandomForestClassifier(criterion="gini", min_samples_leaf=5,
                                    n_estimators=nTrees[c], max_features=max_ftrs[c], random_state=iter))])
            pipeKNN=Pipeline(steps=[('KNN', KNeighborsClassifier(n_neighbors=knn[c],metric=knn_metric[c]))])
            pipeLDA=Pipeline(steps=[('LDA',LinearDiscriminantAnalysis(solver=lda_solver[c], ))])
            pipeQDA=Pipeline(steps=[('QDA', QuadraticDiscriminantAnalysis(reg_param=0.01))])
            pipeLSVC=Pipeline(steps=[('LSVC', SVC(kernel="linear", C=C_Lin[c], probability=True, 
                                                  random_state=iter))])
            pipeRSVC=Pipeline(steps=[('RSVC', SVC(kernel='rbf', C=C_RBF[c], gamma=svd_gamma[c], 
                                                  probability=True, random_state=iter))])
            pipeGMLVQ=Pipeline(steps=[('GMLVQ',GMLVQ(distance_type='adaptive-squared-euclidean',
                solver_type=s_glob[keyD],random_state=iter,prototype_n_per_class=g_num_prots_all[keyD], 
                relevance_regularization=reg_comp[c],activation_type=act_g[keyD],
                relevance_n_components=int(rel_comp_gmlvq[keyD])))])
            pipeLGMLVQ1=Pipeline(steps=[('LGMLVQ1', LGMLVQ(distance_type='local-adaptive-squared-euclidean', 
                  random_state=iter,prototype_n_per_class=l_num_prots_all1[keyD], activation_type=act_l1[keyD],
                  relevance_localization=rel_loc,solver_type=s_loc1[keyD],
                  relevance_n_components=int(rel_comp_lgmlvq1[keyD])))])   
            pipeLGMLVQ2=Pipeline(steps=[('LGMLVQ2', LGMLVQ(distance_type='local-adaptive-squared-euclidean', 
                random_state=iter,prototype_n_per_class=l_num_prots_all2[keyD], activation_type=act_l2[keyD], 
                solver_type=s_loc2[keyD],relevance_n_components=int(rel_comp_lgmlvq2[keyD]),
                relevance_localization=rel_loc))])   
            pipeGNB=Pipeline(steps=[('GNB', GaussianNB())])
            pipeLogLASSO=Pipeline(steps=[('LogLASSO',LogisticRegression(C=C_LR[c], solver='saga',
                            class_weight='balanced',penalty='l1', random_state=iter))])
            pipeClassifiers_all={'RF':pipeRF, 'KNN':pipeKNN, 'LDA':pipeLDA, 'QDA': pipeQDA, 'LSVC': pipeLSVC, 
                      'RSVC': pipeRSVC, 'GMLVQ': pipeGMLVQ, 'LGMLVQ1': pipeLGMLVQ1, 'LGMLVQ2': pipeLGMLVQ2, 
                      'GNB': pipeGNB,'LogLASSO':pipeLogLASSO}
            for keyClf, clf in pipeClassifiers_all.items():
                clf.fit(X, Y)
                clf_all[str(iter)+'-'+keyClf]=clf
                if keyClf in use_permutation_imp:
                    permimp = permutation_importance(clf, X, Y, n_repeats=nreps, scoring=make_scorer(lowest_cwacc))
                    permut_model[str(iter)+'-'+keyClf]=permimp.importances
                if keyClf in ['GMLVQ', 'LGMLVQ1', 'LGMLVQ2']:    
                    all_prots[str(iter)+'-'+keyClf]=clf[keyClf].prototypes_ 
        print('Training final %s Classification models complete'%dataset_type)
        pipe_per_clf, permut_per_clf, prots_per_clf={},{},{}
        for temp_clf in pipeClassifiers_all.keys():
            temp_per_clf, temp_permut_per_clf, temp_prots =[],[],[]
            for tempK, tempV in clf_all.items():    
                itr, keyClf_temp=tempK.split('-')
                if keyClf_temp==temp_clf:
                    temp_per_clf.append(tempV)
                    if temp_clf in use_permutation_imp: 
                        temp_permut_clf=permut_model[tempK]
                        temp_permut_per_clf.append(temp_permut_clf)
                    if temp_clf in ['GMLVQ', 'LGMLVQ1', 'LGMLVQ2']:  
                        temp_prots.append(all_prots[tempK])
            pipe_per_clf[temp_clf]=temp_per_clf.copy()
            if temp_clf in use_permutation_imp:
                permut_per_clf[temp_clf]=temp_permut_per_clf.copy()
            if temp_clf in ['GMLVQ', 'LGMLVQ1', 'LGMLVQ2']:
                prots_per_clf[temp_clf]=temp_prots.copy()                  
        del pipeClassifiers_all, temp_per_clf, temp_prots, temp_permut_per_clf
        with open(modelsClassify, 'wb') as f:  # open a text file
            pickle.dump(pipe_per_clf, f) 
        with open('%s'%modelsClassify_permut, 'wb') as f:  # open a text file
            pickle.dump(permut_per_clf, f) 
        modelsClassify_prots='%s%s_%s.pkl'%(savepicklpath,dataset_type, 'Prots')  
        with open('%s'%modelsClassify_prots, 'wb') as f:  # open a text file
            pickle.dump(prots_per_clf, f) 
        print('Saving final %s models complete'%dataset_type)
        del clf_all, permut_model
    c+=1

Training final Core-DT-5Cls Classification models complete
Saving final Core-DT-5Cls models complete
Training final Core-DT-onlyED Classification models complete
Saving final Core-DT-onlyED models complete
Training final Core-DT-3ED-Others Classification models complete
Saving final Core-DT-3ED-Others models complete
Training final Core-DT-EDEQ-5Cls Classification models complete
Saving final Core-DT-EDEQ-5Cls models complete
Training final Core-DT-EDEQ-only-ED Classification models complete
Saving final Core-DT-EDEQ-only-ED models complete
Training final Core-DT-EDEQ-3ED-Others Classification models complete
Saving final Core-DT-EDEQ-3ED-Others models complete
Training final Core-5Cls Classification models complete
Saving final Core-5Cls models complete
Training final Core-onlyED Classification models complete
Saving final Core-onlyED models complete
Training final Core-3ED-Others Classification models complete
Saving final Core-3ED-Others models complete


# Regression

In [40]:
# %autoreload 2
#import importlib
#importlib.reload(
from HypOptRegress import GridSearchRegressors
choice_regs=[1.0, 3.0]
for keyD, val in choice_dict.items():
    if keyD in choice_regs:    
        if keyD>=3:        
            X=dataset_types[keyD]['zXtrain']  
            param_grid={'RF_Max_Features': [5,7,10,16], 'RF_n_Trees':[100, 300, 500], 'RF_min_leaf':5,
           'scorer_name':'Lowest_CWA', 'LRL1_C': np.array([0.001, 0.01, 0.1, 1])}
        else:
            X=dataset_types[keyD]['mice_zXtrain']
            param_grid={'RF_Max_Features': [5,7,10, 20,25], 'RF_n_Trees':[100, 300, 500], 'RF_min_leaf':5,
           'scorer_name':'Lowest_CWA', 'LRL1_C': np.array([0.001, 0.01, 0.1, 1])}
        Y=dataset_types[keyD]['Ytrain_reg']   
        print(keyD)
        param_grid_reg={'scorer': 'neg_root_mean_squared_log_error', 'RF':param_grid}
        GridSearchRegressors(X, Y, param_grid_reg)

1.0
KNN: 
    K Dist Metric     Score  Abs_Score
3  3      cosine -0.250908   0.250908
1  5   minkowski -0.252987   0.252987
2  7   minkowski -0.253428   0.253428
5  7      cosine -0.257942   0.257942
RF: 
     n_Trees  Max_Features       Criterion     Score  Abs_Score
27      100            25  absolute_error -0.236910   0.236910
59      500            25         poisson -0.237573   0.237573
58      300            25         poisson -0.237780   0.237780
56      500            20         poisson -0.238342   0.238342
Linear SVM: 
        C     Score  Abs_Score
2    0.1 -0.227737   0.227737
3    1.0 -0.231015   0.231015
4   10.0 -0.231967   0.231967
5  100.0 -0.231985   0.231985
RBF SVM: 
         C   Gamma     Score  Abs_Score
31  100.0  0.0010 -0.225034   0.225034
26   10.0  0.0100 -0.225551   0.225551
25   10.0  0.0010 -0.242093   0.242093
30  100.0  0.0001 -0.242156   0.242156
3.0
KNN: 
    K Dist Metric     Score  Abs_Score
1  5   minkowski -0.228309   0.228309
2  7   minkowski -0.2

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, root_mean_squared_error

In [75]:
regressor_all.keys()

dict_keys(['RF', 'KNN', 'LSVC', 'RSVC', 'LASSOReg'])

In [76]:
knn_met, knn_k=['cosine', 'minkowski'], [3,5]
rf_tree,rf_ftr, rf_crit=[500,100],[20,7],['poisson', 'squared_error']
lsvm, c=[0.1,0.01], 0
rsvm_c, rsvm_g=[100, 10],[0.001,0.01]
iters=5
for keyD, val in choice_dict.items():
    if keyD in choice_regs:    
        dataset_type=save_dict[keyD]#'Core_DT'#save_dict[choice]
        model_fname='%s_reg2.pkl'%dataset_type
        modelsRegress='%s%s'%(savepicklpath, model_fname)          
        Ytrain, Ytest=dataset_types[keyD]['Ytrain_reg'], dataset_types[keyD]['Ytest_reg']               
        if keyD>=3: 
            Xtrain, Xtest=dataset_types[keyD]['zXtrain'], dataset_types[keyD]['zXtest']
        else:
            Xtrain, Xtest=dataset_types[keyD]['mice_zXtrain'], dataset_types[keyD]['mice_zXtest']
        if os.path.exists(modelsRegress):
            print('Pickled files found')#pickle.load(open('%s'%modelsClassify_permut, "rb"))
            regressor_all=pickle.load(open('%s'%modelsRegress, "rb"))
            for keyReg in regressor_all.keys():            
                perf_dict=regressor_all[keyReg]['Performance']
                for split in ['Training', 'Test']:
                    print('Exp-%d%s %s: R2= %.3f (%.3f), RMSE= %.3f (%.3f)'%(keyD, keyReg, split,
                                    perf_dict[keyReg+split]['R2(mean)'], perf_dict[keyReg+split]['R2(std)'],
                                    perf_dict[keyReg+split]['RMSE(mean)'], perf_dict[keyReg+split]['RMSE(std)']))
        else:
            print('Pickled files not present yet')
            reg_all, regressor_all, perf_dict={},{},{}
            tr_perf, te_perf=np.zeros((2, iters)), np.zeros((2, iters))
            train_perf_all, test_perf_all={},{}            
            pipeRF=Pipeline(steps=[('RF', RandomForestRegressor(criterion=rf_crit[c], min_samples_leaf=5,
                                        n_estimators=rf_tree[c], max_features=rf_ftr[c]))])
            pipeKNN=Pipeline(steps=[('KNN', KNeighborsRegressor(n_neighbors=knn_k[c],metric=knn_met[c]))])                        
            pipeLSVR=Pipeline(steps=[('LinSVC', SVR(kernel="linear", C=lsvm[c]))])
            pipeRSVR=Pipeline(steps=[('RbfSVC', SVR(kernel='rbf', C=rsvm_c[c], gamma=rsvm_g[c]))])
            pipeLASSO=Pipeline(steps=[('LASSO', LassoCV(cv=5))])
            pipeRegressors={'RF':pipeRF, 'KNN':pipeKNN, 'LSVC': pipeLSVR, 'RSVC': pipeRSVR,
                            'LASSOReg': pipeLASSO}            
            for keyReg, reg in pipeRegressors.items():
                for iter in range(iters):
                    reg.fit(Xtrain, Ytrain)
                    reg_all[str(iter)+'-'+keyReg]=reg
                    train_pred_labs_eds=reg.predict(Xtrain)
                    test_pred_labs_eds=reg.predict(Xtest)
                    tr_perf[0,iter]=r2_score(Ytrain, train_pred_labs_eds)
                    te_perf[0,iter]=r2_score(Ytest, test_pred_labs_eds)
                    tr_perf[1,iter]=root_mean_squared_error(Ytrain, train_pred_labs_eds)
                    te_perf[1,iter]=root_mean_squared_error(Ytest, test_pred_labs_eds)                
                train_perf_all[keyReg], test_perf_all[keyReg]=tr_perf, te_perf
                perf_dict[keyReg+'Training']={'R2(mean)': np.mean(tr_perf[0,:]),'R2(std)': np.std(tr_perf[0,:]),
                                'RMSE(mean)': np.mean(tr_perf[1,:]),'RMSE(std)': np.std(tr_perf[1,:])}
                perf_dict[keyReg+'Test']={'R2(mean)': np.mean(te_perf[0,:]),'R2(std)': np.std(te_perf[0,:]),
                                      'RMSE(mean)': np.mean(te_perf[1,:]),'RMSE(std)': np.std(te_perf[1,:])}
                regressor_all[keyReg]={'Models': reg_all, 'Performance': perf_dict}
                for split in ['Training', 'Test']:
                    print('Exp-%d%s %s: R2= %.3f (%.3f), RMSE= %.3f (%.3f)'%(keyD, keyReg, split,
                                perf_dict[keyReg+split]['R2(mean)'], perf_dict[keyReg+split]['R2(std)'],
                                perf_dict[keyReg+split]['RMSE(mean)'], perf_dict[keyReg+split]['RMSE(std)']))
            with open(modelsRegress, 'wb') as f:  # open a text file
                pickle.dump(regressor_all, f)  
        

Pickled files found
Exp-1RF Training: R2= 0.800 (0.001), RMSE= 0.560 (0.001)
Exp-1RF Test: R2= 0.659 (0.004), RMSE= 0.826 (0.004)
Exp-1KNN Training: R2= 0.751 (0.000), RMSE= 0.625 (0.000)
Exp-1KNN Test: R2= 0.452 (0.000), RMSE= 1.047 (0.000)
Exp-1LSVC Training: R2= 0.614 (0.000), RMSE= 0.778 (0.000)
Exp-1LSVC Test: R2= 0.495 (0.000), RMSE= 1.005 (0.000)
Exp-1RSVC Training: R2= 0.686 (0.000), RMSE= 0.702 (0.000)
Exp-1RSVC Test: R2= 0.522 (0.000), RMSE= 0.978 (0.000)
Exp-1LASSOReg Training: R2= 0.593 (0.000), RMSE= 0.800 (0.000)
Exp-1LASSOReg Test: R2= 0.634 (0.000), RMSE= 0.856 (0.000)
Pickled files found
Exp-3RF Training: R2= 0.804 (0.001), RMSE= 0.536 (0.002)
Exp-3RF Test: R2= 0.628 (0.001), RMSE= 0.820 (0.001)
Exp-3KNN Training: R2= 0.726 (0.000), RMSE= 0.634 (0.000)
Exp-3KNN Test: R2= 0.369 (0.000), RMSE= 1.068 (0.000)
Exp-3LSVC Training: R2= 0.584 (0.000), RMSE= 0.781 (0.000)
Exp-3LSVC Test: R2= 0.606 (0.000), RMSE= 0.844 (0.000)
Exp-3RSVC Training: R2= 0.634 (0.000), RMSE= 0.733 (